In [1]:
import os
import sys
import time

import numpy as np

sys.path.insert(0, os.path.abspath('..'))
from evolutron.tools import load_dataset, none2str, Handle, shape, get_args
from evolutron.engine import Model
from evolutron.templates import callback_templates as cb

import keras.backend as K
from keras.layers import Convolution1D, MaxPooling1D, Dense, Flatten, Reshape
from keras.layers import Input
from keras.layers import BatchNormalization
from evolutron.extra_layers import Dedense, Upsampling1D, Deconvolution1D

from keras.metrics import categorical_accuracy
from evolutron.extra_metrics import mean_cat_acc

from keras.losses import mean_squared_error, categorical_crossentropy

from keras.utils.np_utils import to_categorical
    
seed = 7
np.random.seed(seed)

Using Theano backend.


In [2]:
dataset = load_dataset('ecoli',padded=True)
input_shape = (100, 20)
seq_length, alphabet = input_shape
filters = 15
filter_length = 20

Dataset size: 4306


In [3]:
# Input LayerRO
inp = Input(shape=input_shape, name='aa_seq')

# Convolutional Layers
conv = Convolution1D(filters=filters,
                       kernel_size=filter_length,
                       kernel_initializer='glorot_uniform',
                       activation='relu',
                       padding='same',
                       name='Conv')(inp)


max_pool = MaxPooling1D(pool_size=seq_length)(conv)
flat = Flatten()(max_pool)

# Fully-Connected encoding layers
encoded = Dense(filters,
                kernel_initializer='glorot_uniform',
                activation='sigmoid',
                name='FCEnc')(flat)


fc_dec = Dedense(encoded._keras_history[0],
                  activation='linear',
                  name='FCDec')(encoded)



unflat = Reshape(max_pool._keras_shape[1:])(fc_dec)


upsample = Upsampling1D(max_pool._keras_history[0].input_shape[1], name='Upsampling')(unflat)

decoded = Deconvolution1D(conv._keras_history[0],
                          apply_mask=False,
                          activation='sigmoid',
                          name='Deconv')(upsample)

model = Model(inputs=inp, outputs=decoded, name='DeepCoDER', classification=False)

losses = [mean_squared_error]

# Metrics
metrics = [mean_cat_acc]

# Compilation

model.compile(optimizer='nadam',
              loss=losses,
              metrics=metrics,
              lr=.002)

model.display_network_info()

Neural Network has 6290 trainable parameters
  #  Name             Shape      Parameters
---  ---------------  -------  ------------
  0  aa_seq           100x20              0
  1  Conv             100x15           6015
  2  max_pooling1d_1  1x15                0
  3  flatten_1        15                  0
  4  FCEnc            15                240
  5  FCDec            15                 15
  6  reshape_1        1x15                0
  7  Upsampling       100x15              0
  8  Deconv           100x20             20


## Testing save and load on untrained model

In [4]:
model.save('test')

Model saved to: test


In [5]:
from evolutron.extra_layers import custom_layers
from evolutron.engine import load_model

In [7]:
m = load_model('test', custom_objects=custom_layers, compile=False)